Необходимо используя python проставить отрасли для 20000 тысяч компаний из файла df_nodes_test_publ.csv


Формат Node,Y <Node>,<Industry>

Пример Node,Y 1,32 3,34 …

df_edges_orig.csv – факт платежа между двумя компаниями, перевод идет от NodeLeft к NodeRight

* NodeLeft – обезличенный инн первой компании;

* NodeRight – обезличенный инн второй компании;

* Feat1 – обезличенное кол-во переводов;

* Feat2 – обезличенная сумма переводов;

df_nodes – табличка с клиентами

* Node – обезличенный инн компании;

* Feat3 – Уточнение отрасли, проставлено только для обучающей выборки, можно использовать, как признак, но не нужно предсказывать;

* Feat4 – Регион;

* Y – отрасль, таргет переменная, её нужно предсказать на тестовом наборе;

## Подготовка данных

#### Импортируем нужные библиотеки

In [1]:
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier


In [2]:
# С этими функциями считалось 30 мин :)
'''def final_score(nodes,arr):
    res=np.array([])
    for i in nodes:
        res=np.append(res,get_best_industry(i,arr))
    return res'''


'''def get_best_industry(node, arr):   
    industries = [i for (y, i) in arr if node == y]
    if len(industries)==0: industries=[0] 
    
    return max(Counter(industries).items(),key=(lambda x: x[1]))[0]'''


# А с этими за секунду
#Функция возвращает словарь с организацией и наиболее подходящей отраслью (которая чаще всего повторяется)
#args: 
#  intermediate_result-двумерный массив с номером организации и предсказанной ораслью
def get_best_industry(intermediate_result):
    res={}
    for i,j in intermediate_result:           
        if i in res:
            res[i].append(j)
        else:
            res[i]=[j]
    for i,j in res.items():
        res[i]=max(Counter(j).items(),key=(lambda x: x[1]))[0]
    return res

#Функция возвращает массив подходящих отраслей для введенных организаций 
#args: 
#  nodes-список организация для которых ищутся отрасли
#  intermediate_result-двумерный массив с номером организации и предсказанной ораслью
#return: res-массив подходящих отраслей
def final_score(nodes,intermediate_result):
    res=np.array([])
    res_dict=get_best_industry(intermediate_result)
    for i in nodes:
        if i in res_dict:
            res=np.append(res,res_dict[i])
        else:
            res=np.append(res,0)
    return res


#### Считываем данные файлов в датафреймы


In [3]:
edges = pd.read_csv('df_edges_orig.csv', sep=',')
nodes = pd.read_csv('df_nodes_train.csv', sep=',')
nodes_test = pd.read_csv('df_nodes_test_publ.csv', sep=',')

#### Подготавливаем датафреймы

In [4]:
nodes.columns=["Node","Industry","Region","Y"]
nodes_test.columns=["Node","Region"]
edges.columns=["NodeLeft","NodeRight","Quantity","Amount"]
#Исключаем переводы у которых сумма 0 (они не несут информации)
edges=edges[edges['Amount']!=0]
edges.drop_duplicates(inplace=True)

#### Бьем обучающую выборку на train и valid

In [5]:
nodes_train, nodes_valid, y_train, y_valid = train_test_split(nodes[["Node","Region"]],nodes['Y'],test_size=0.3,random_state=17)

Разделяем датафрейм транзакций для обучения и тестирования (для совместного использования с соответствующими датафреймами организаций)

In [6]:
#В обучающей выборке транзакций должны быть только транзакции у которых с обоих сторон организации из train, чтобы не было
#лишних неинформативных данных
edges_train=edges[edges['NodeLeft'].isin(nodes['Node'].values) & edges['NodeRight'].isin(nodes['Node'].values)]
#Выбираем только те строки в которых есть организации из тестовой выборки и отрезаем часть в которой с обоих сторон 
#тестовые организации, т. к. по ним нет данных по отрасли
edges_test_to=edges[edges['NodeLeft'].isin(nodes_test['Node'].values) & \
                ~(edges['NodeLeft'].isin(nodes_test['Node'].values) & edges['NodeRight'].isin(nodes_test['Node'].values))]
edges_test_from=edges[edges['NodeRight'].isin(nodes_test['Node'].values) & \
                ~(edges['NodeLeft'].isin(nodes_test['Node'].values) & edges['NodeRight'].isin(nodes_test['Node'].values))]

### Финальные данные

#### Создание полной обучающей выборки

In [7]:
#Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции
nodes=nodes[["Node","Region","Y"]]
X_full_train=pd.merge(edges_train, nodes, how='inner',left_on=['NodeLeft'], right_on=['Node'])
X_full_train=pd.merge(X_full_train, nodes, how='inner',left_on=['NodeRight'], right_on=['Node'])

In [8]:
#Добавляем параметр направления транзакции (1 - перевод от целевой организации)
X_full_train['from/to']=1
#Называем столбцы и формируем необходимый порядок
X_full_train.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_full_train.columns=['target_node','ad_node','quantity','amount','target_region','target_Y','ad_region','ad_Y','from/to']
X_full_train=X_full_train[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y','target_Y']]

In [9]:
#Формируем перевернутую таблицу, чтобы учесть для организаций перевод как в одну сторону, так и в другую
X_full_train_ad=X_full_train[['ad_node','from/to','target_node','quantity','amount','ad_region','target_region','target_Y','ad_Y']].copy()
X_full_train_ad.columns=['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y','target_Y']
#Добавляем параметр направления транзакции (0 - перевод в целевую организацию)
X_full_train_ad['from/to']=0

In [10]:
#Собираем вместе финальную таблицу и формируем X и y
X_full_train=pd.concat([X_full_train,X_full_train_ad])
y_full_train=X_full_train['target_Y']
X_full_train=X_full_train.drop(['target_Y'],axis=1)

In [11]:
#Обозначаем категориальные столбцы
for c in ['from/to', 'ad_node','target_region', 'ad_region', 'ad_Y']:
    X_full_train[c].astype('category')


#### Создание тестовой выборки

In [12]:
##Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции, но с одной стороны из валидации
#с другой из train
X_test_to=pd.merge(edges_test_to, nodes_test, how='inner',left_on=['NodeLeft'], right_on=['Node'])
X_test_to=pd.merge(X_test_to, nodes, how='inner',left_on=['NodeRight'], right_on=['Node'])

In [13]:
#Добавляем параметр направления транзакции (1 - перевод от целевой организации)
X_test_to['from/to']=1
#Называем столбцы и формируем необходимый порядок
X_test_to.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_test_to.columns=['target_node','ad_node','quantity','amount','target_region','ad_region','ad_Y','from/to']
X_test_to=X_test_to[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y']]

In [14]:
##Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции, но с одной стороны из валидации
#с другой из train (теперь наоборот)
X_test_from=pd.merge(edges_test_from, nodes_test, how='inner',left_on=['NodeRight'], right_on=['Node'])
X_test_from=pd.merge(X_test_from, nodes, how='inner',left_on=['NodeLeft'], right_on=['Node'])

In [15]:
#Добавляем параметр направления транзакции (0 - перевод в целевую организацию)
X_test_from['from/to']=0
#Называем столбцы и формируем необходимый порядок
X_test_from.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_test_from.columns=['ad_node','target_node','quantity','amount','target_region','ad_region','ad_Y','from/to']
X_test_from=X_test_from[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y']]

In [16]:
#Собираем вместе финальную таблицу
X_test=pd.concat([X_test_to,X_test_from])

#Обозначаем категориальные столбцы
for c in ['from/to', 'ad_node','target_region', 'ad_region', 'ad_Y']:
    X_test[c].astype('category')

### Валидационные данные

#### Создание обучающей выборки



In [17]:
#Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции
nodes_train=pd.concat([nodes_train,y_train],axis=1)
X_train=pd.merge(edges_train, nodes_train, how='inner',left_on=['NodeLeft'], right_on=['Node'])
X_train=pd.merge(X_train, nodes_train, how='inner',left_on=['NodeRight'], right_on=['Node'])

In [18]:
#Добавляем параметр направления транзакции (1 - перевод от целевой организации)
X_train['from/to']=1
#Называем столбцы и формируем необходимый порядок
X_train.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_train.columns=['target_node','ad_node','quantity','amount','target_region','target_Y','ad_region','ad_Y','from/to']
X_train=X_train[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y','target_Y']]

In [19]:
#Формируем перевернутую таблицу, чтобы учесть для организаций перевод как в одну сторону, так и в другую
X_train_ad=X_train[['ad_node','from/to','target_node','quantity','amount','ad_region','target_region','target_Y','ad_Y']].copy()
X_train_ad.columns=['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y','target_Y']
#Добавляем параметр направления транзакции (0 - перевод в целевую организацию)
X_train_ad['from/to']=0

In [20]:
#Собираем вместе финальную таблицу и формируем X и y
X_train=pd.concat([X_train,X_train_ad])
y_train=X_train['target_Y']
X_train=X_train.drop(['target_Y'],axis=1)

In [21]:
#Обозначаем категориальные столбцы
for c in ['from/to', 'ad_node','target_region', 'ad_region', 'ad_Y']:
    X_train[c].astype('category')


#### Создание валидационной выборки

In [22]:
#Собираем датафреймы транзакций для валидации аналогично тестовой части
edges_valid_to=edges_train[edges_train['NodeLeft'].isin(nodes_valid['Node'].values) & \
                ~(edges_train['NodeLeft'].isin(nodes_valid['Node'].values) & edges_train['NodeRight'].isin(nodes_valid['Node'].values))]
edges_valid_from=edges_train[edges_train['NodeRight'].isin(nodes_valid['Node'].values) & \
                ~(edges_train['NodeLeft'].isin(nodes_valid['Node'].values) & edges_train['NodeRight'].isin(nodes_valid['Node'].values))]

In [23]:
##Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции, но с одной стороны из валидации
#с другой из train
X_valid_to=pd.merge(edges_valid_to, nodes_valid, how='inner',left_on=['NodeLeft'], right_on=['Node'])
X_valid_to=pd.merge(X_valid_to, nodes_train, how='inner',left_on=['NodeRight'], right_on=['Node'])

In [24]:
#Добавляем параметр направления транзакции (1 - перевод от целевой организации)
X_valid_to['from/to']=1
#Называем столбцы и формируем необходимый порядок
X_valid_to.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_valid_to.columns=['target_node','ad_node','quantity','amount','target_region','ad_region','ad_Y','from/to']
X_valid_to=X_valid_to[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y']]

In [25]:
##Создаем объединенную таблицу, присоединяя отрасли и регионы для обоих участинков транзакции, но с одной стороны из валидации
#с другой из train (теперь наоборот)
X_valid_from=pd.merge(edges_valid_from, nodes_valid, how='inner',left_on=['NodeRight'], right_on=['Node'])
X_valid_from=pd.merge(X_valid_from, nodes_train, how='inner',left_on=['NodeLeft'], right_on=['Node'])

In [26]:
#Добавляем параметр направления транзакции (0 - перевод в целевую организацию)
X_valid_from['from/to']=0
#Называем столбцы и формируем необходимый порядок
X_valid_from.drop(['Node_x','Node_y'],axis=1,inplace=True)
X_valid_from.columns=['ad_node','target_node','quantity','amount','target_region','ad_region','ad_Y','from/to']
X_valid_from=X_valid_from[['target_node','from/to','ad_node','quantity','amount','target_region','ad_region','ad_Y']]

In [27]:
#Собираем вместе финальную таблицу
X_validation=pd.concat([X_valid_to,X_valid_from])


In [28]:
#Обозначаем категориальные столбцы
for c in ['from/to', 'ad_node','target_region', 'ad_region', 'ad_Y']:
    X_validation[c].astype('category')

# Обучение - валидация

### Обучаем дерево
 
  

In [48]:
# В итоге получились такие параметры
rfc = RandomForestClassifier(random_state=17, n_jobs=-1, n_estimators=100,max_depth=20,max_features=2,min_samples_leaf=2) #class_weight='balanced'


#### Подбираем параметры

In [31]:
#max_depth_values = [5, 10, 15, 20, 25]
max_features_values = [4, 5, 6, 7]
tree_params = {#'max_depth': max_depth_values,
               'max_features': max_features_values}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [32]:
gcv = GridSearchCV(rfc, tree_params, scoring='accuracy', n_jobs=-1, cv=skf, verbose=1)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 14.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': [4, 5, 6, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='accuracy', verbose=1)

In [34]:
gcv.best_params_

0.53178293319964098

#### Валидируем наше решение

In [49]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=130, n_jobs=-1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [1]:
y_valid_result=rfc.predict(X_validation)


NameError: name 'rfc' is not defined

In [ ]:
# Приводим промежуточный результат к двумерному массиву
intermediate_result=X_validation['target_node'].values
y_valid_result=np.array(y_valid_result)
intermediate_result=np.dstack((intermediate_result,y_valid_result))
intermediate_result=np.reshape(intermediate_result,(-1,2))

In [ ]:
# С помощью функции получаем итоговые предсказания, которые можно сравнивать 
final=final_score(nodes_valid['Node'].values,intermediate_result)

In [47]:
#Получаем долю верных ответов 
accuracy_score(y_valid.values,final.astype(int))

0.20443228038164746

#### К сожеланию, удалось получить долю верных ответов на отложенной выборке только 20%

### Используем catboost

#### Пытаемся обучить catboost

In [34]:
model = CatBoostClassifier(random_seed=17,loss_function='MultiClass',iterations=2, learning_rate=1, depth=2)
model.fit(X_train, y_train, cat_features=[1,2,5,6,7])

In [30]:
pred_class = model.predict(X_validation)

In [31]:
# Приводим промежуточный результат к двумерному массиву
intermediate_result=X_validation['target_node'].values
pred_class=np.reshape(pred_class,(-1))
intermediate_result=np.dstack((intermediate_result,pred_class))
intermediate_result=np.reshape(intermediate_result,(-1,2))

In [32]:
final_cat=final_score(nodes_valid['Node'].values,intermediate_result)

In [33]:
accuracy_score(y_valid.values,final_cat.astype(int))

0.050860987569848332

#### С catboost получилось еще хуже, максимум показывало долю 5% (видимо что-то делаю не так)

## Финальное обучение

#### Исходя из предыдущих результатов я выбрал финальный алгоритм на деревьях

In [86]:
f_rfc = RandomForestClassifier(random_state=17, n_jobs=-1, n_estimators=100,max_depth=20,max_features=2,min_samples_leaf=2)

In [87]:
f_rfc.fit(X_full_train,y_full_train)
f_y_valid_result=f_rfc.predict(X_test)

In [88]:
# Приводим промежуточный результат к двумерному массиву
f_intermediate_result=X_test['target_node'].values
f_y_valid_result=np.array(f_y_valid_result)
f_intermediate_result=np.dstack((f_intermediate_result,f_y_valid_result))
f_intermediate_result=np.reshape(f_intermediate_result,(-1,2))

In [89]:
# С помощью функции получаем итоговые предсказания, которые можно сравнивать 
f_final=final_score(nodes_test['Node'].values,f_intermediate_result)

In [90]:
#Формируем датафрейм, чтобы выгрузить в csv
f_test = pd.read_csv('df_nodes_test_publ.csv', sep=',')
f_test.drop(['Feat4'],axis=1,inplace=True)

In [91]:
f_test['Y']=f_final.astype(int)
f_test.to_csv('SakharovDmitriy-24092017-SberbankIndustry.csv', index=False, header=True)